# Music of big cities

**Comparison of Moscow and St. Petersburg: myths and analysis**

For example:
1. Moscow is a metropolis driven by the intense rhythm of the workweek.  
2. St. Petersburg is the cultural capital, with its distinct tastes.  

Using data from Yandex.Music, you will analyze user behavior in these two cities.

**Research goal**
Test the following hypotheses:  
1. User activity depends on the day of the week, and this pattern differs between Moscow and St. Petersburg.  
2. On Monday mornings, different music genres dominate in Moscow compared to St. Petersburg. Similarly, on Friday evenings, genre preferences vary by city.  
3. Moscow and St. Petersburg have distinct genre preferences: pop music is more popular in Moscow, while Russian rap is more common in St. Petersburg.  

**Research process**  

User behavior data will be sourced from the file `yandex_music_project.csv`. The quality of the data is unknown, so an initial data overview will be necessary.  

You will:  
1. Review the data for errors and assess their impact on the research.  
2. During preprocessing, correct the most critical data issues if possible.  

**Research steps**  
1. Data overview.  
2. Data preprocessing.  
3. Hypotheses testing.

Load libraries:

In [47]:
import pandas as pd
import plotly.express as px

from caseconverter import snakecase
from collections import defaultdict
from IPython.display import display

In [8]:
FIG_WIDTH = 10 * 100
FIG_HEIGHT = 5 * 100

In [71]:
def get_statistics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generate summary statistics for each column in a DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        pd.DataFrame: DataFrame with column statistics, including column name, data type,
                      count of unique values, a sample of unique values, null count, and
                      percentage of null values.
    """
    rows_total = len(df)
    
    stats_list = [
        {
            'column_name': f"'{column}'",
            'data_type': str(df[column].dtype),
            'count_unique': df[column].nunique(),
            'sample_values': (
                pd.Series(df[column].dropna().unique())
                .sample(min(5, len(df[column].dropna().unique())))
                .apply(lambda x: round(x, 2) if pd.api.types.is_numeric_dtype(df[column]) else x)
                .tolist()
            ),
            'count_null': df[column].isnull().sum(),
            'pct_null': round((df[column].isnull().sum() / rows_total) * 100, 0),
        }
        for column in df.columns
    ]

    # Convert the list of statistics to a DataFrame
    print(f"Dataframe size: {df.shape[0]} rows x {df.shape[1]} columns")
    print(f"Full duplicate rows: {df.duplicated().sum()}")
    return pd.DataFrame(stats_list)

In [93]:
def plot_unique_counts(df: pd.DataFrame, columns: list, n_cols: int = 1, fig_size: tuple = (800, 600)):
    """
    Generate a Plotly bar chart for unique value counts in selected columns, 
    arranged in a grid with configurable rows and columns.

    Args:
        df (pd.DataFrame): Input DataFrame.
        columns (list): List of column names to analyze for unique counts.
        n_cols (int): Number of columns in the facet grid. Defaults to 1.
        fig_size (tuple): Size of the figure (width, height). Defaults to (800, 600).

    Returns:
        None: Displays the Plotly bar chart.
    """
    # Prepare data for plotting
    data_list = []
    for column in columns:
        df_temp = (
            df[column]
            .value_counts()
            .reset_index()
            .set_axis(['values', 'ucount'], axis=1)
            .assign(column=column)
        )
        data_list.append(df_temp)
    
    df_combined = pd.concat(data_list, ignore_index=True)

    # Create the bar chart
    fig = px.bar(
        df_combined.sort_values('ucount', ascending=True),
        x='ucount',
        y='values',
        color='column',
        facet_col='column',
        title='Unique counts of values in selected columns',
        template='plotly_white',
        width=fig_size[0], height=fig_size[1],
        facet_col_wrap=n_cols
    )

    # Set Y-axis scale to be specific for each facet
    fig.update_layout(
        xaxis_title='Unique count',
        yaxis_title='Values',
        showlegend=False
    )
    
    # Make the Y-axis independent for each facet
    fig.for_each_yaxis(lambda axis: axis.update(matches=None))

    fig.show()

In [74]:
try:
    raw_music = pd.read_csv('yandex_music_project.csv')
except:
    raw_music = pd.read_csv('/datasets/yandex_music_project.csv')

## Data overview

Let's get an initial understanding of the Yandex.Music data.

In [75]:
display(raw_music.head(5))

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [42]:
get_statistics(raw_music)

Dataframe size: 65079 rows x 7 columns
Full duplicate rows: 3826


,column_name,data_type,count_unique,sample_values,count_null,pct_null
0,' userID',object,41748,"[6929C205, CC89C9A7, 2B1212DE, 77F8C592, 12692...",0,0.0
1,'Track',object,47245,"[Where The Wind Blows, Ak Kaen & Ой то не вече...",1231,2.0
2,'artist',object,43605,"[Just Go feat. НэВ, Rain Diary, Jonny Craig, B...",7203,11.0
3,'genre',object,289,"[southern, lovers, dirty, salsa, popdance]",1198,2.0
4,' City ',object,2,"[Moscow, Saint-Petersburg]",0,0.0
5,'time',object,20392,"[08:15:34, 08:06:05, 13:08:22, 20:14:41, 21:27...",0,0.0
6,'Day',object,3,"[Monday, Wednesday, Friday]",0,0.0


In [ ]:
raw_music.columns

In [98]:
plot_unique_counts(
    raw_music.rename(columns=snakecase), 
    ['genre', 'day', 'city'],
    n_cols=1,
    fig_size=(FIG_WIDTH, FIG_HEIGHT)
)

In [70]:
df_temp = (
    raw_music.genre
    .value_counts()
    .reset_index()
    .set_axis(['values', 'ucount'], 'columns')
    .assign(column='genre')
)

display(df_temp.head(5))

fig = px.bar(df_temp.sort_values('ucount', ascending=True), x='ucount', y='values', title='Unique Counts of Selected Columns', template='plotly_white').show()

C:\Users\micha\AppData\Local\Temp/ipykernel_38980/2718652864.py:2: FutureWarning:

In a future version of pandas all arguments of DataFrame.set_axis except for the argument 'labels' will be keyword-only.



,values,ucount,column
0,pop,8850,genre
1,dance,6761,genre
2,rock,6192,genre
3,electronic,5852,genre
4,hip,3148,genre


The table contains seven columns, all with the `object` data type.

According to the data documentation:
- `userID` — user identifier;
- `Track` — track title;
- `artist` — artist name;
- `genre` — genre name;
- `City` — user's city;
- `time` — listening start time;
- `Day` — day of the week.

Three style issues are noticeable in the column names:
1. Lowercase and uppercase letters are mixed.
2. Spaces are present.
3. Identify another issue with the column names and describe it in this point.

The number of values in the columns varies, indicating there are missing values in the data. There are also full duplicates and inconsistent content in several columns.

**Conclusions**

Each row in the table contains data about a listened track. Some columns describe the track itself, including the title, artist, and genre. The other columns provide information about the user, such as their city and the time they listened to the music.

Preliminary analysis suggests that the data is sufficient to test the hypotheses. However, there are missing values, and the column names do not adhere to good style conventions.

To proceed, these data issues need to be addressed.

In [33]:
df_music = (
    raw_music.copy()
    .rename(columns=snakecase)
    .fillna('unknown')
    .drop_duplicates()
)

In [34]:
get_statistics(df_music)

Dataframe length: 61253
Full duplicate rows: 0


,column_name,data_type,count_unique,sample_values,count_null,pct_null
0,'user_id',object,41748,"[6CFB4754, E75AB6B3, 9488F837, 321D0506, 667A0...",0,0.0
1,'track',object,47246,"[I'm In The House, Our Place, Parvaneh ha(The ...",0,0.0
2,'artist',object,43606,"[Beggars Ball, Amel Bent, Earl Okin, Lounge De...",0,0.0
3,'genre',object,290,"[mandopop, oceania, fitness, rave, techno]",0,0.0
4,'city',object,2,"[Moscow, Saint-Petersburg]",0,0.0
5,'time',object,20392,"[13:12:45, 20:58:21, 20:12:25, 14:54:35, 13:56...",0,0.0
6,'day',object,3,"[Monday, Wednesday, Friday]",0,0.0


Ещё раз посчитайте явные дубликаты в таблице — убедитесь, что полностью от них избавились:

Теперь избавьтесь от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведите на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
* извлеките нужный столбец датафрейма, 
* примените к нему метод сортировки,
* для отсортированного столбца вызовите метод, который вернёт уникальные значения из столбца.

In [16]:
# Просмотр уникальных названий жанров
df.genre.sort_values().unique()

NameError: name 'df' is not defined

Просмотрите список и найдите неявные дубликаты названия `hiphop`. Это могут быть названия с ошибками или альтернативные названия того же жанра.

Вы увидите следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишите функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [17]:
# Функция для замены неявных дубликатов
def replace_wrong_genres( wrong_genres, correct_genres ):
    for counter in range( len( wrong_genres ) ):
        df.genre = df.genre.replace( wrong_genres[counter], correct_genres[counter] )

Вызовите `replace_wrong_genres()` и передайте ей такие аргументы, чтобы она устранила неявные дубликаты: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [18]:
# Устранение неявных дубликатов
replace_wrong_genres( ['hip', 'hop', 'hip-hop'], ['hiphop', 'hiphop', 'hiphop'] )

NameError: name 'df' is not defined

Проверьте, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выведите отсортированный список уникальных значений столбца `genre`:

In [19]:
# Проверка на неявные дубликаты
df.genre.sort_values().unique()

NameError: name 'df' is not defined

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Вы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения вы заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверьте это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделите пользователей Москвы и Санкт-Петербурга
* Сравните, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Для тренировки сначала выполните каждый из расчётов по отдельности. 

Оцените активность пользователей в каждом городе. Сгруппируйте данные по городу и посчитайте прослушивания в каждой группе.



In [20]:
# Подсчёт прослушиваний в каждом городе
df.groupby( 'city' ).user_id.count()

NameError: name 'df' is not defined

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируйте данные по дню недели и подсчитайте прослушивания в понедельник, среду и пятницу. Учтите, что в данных есть информация только о прослушиваниях только за эти дни.


In [21]:
# Подсчёт прослушиваний в каждый из трёх дней
df.groupby( 'day' ).user_id.count()

NameError: name 'df' is not defined

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Вы видели, как работает группировка по городу и по дням недели. Теперь напишите функцию, которая объединит два эти расчёта.

Создайте функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраните в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого примените последовательную фильтрацию с логической индексацией.

Затем посчитайте значения в столбце `user_id` получившейся таблицы. Результат сохраните в новую переменную. Верните эту переменную из функции.

In [22]:
# <создание функции number_tracks()>
# Функция для подсчёта прослушиваний для конкретного города и дня.
# С помощью последовательной фильтрации с логической индексацией она 
# сначала получит из исходной таблицы строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id. 
# Это количество функция вернёт в качестве результата

# Объявляется функция с двумя параметрами: day, city.
def number_tracks( day, city ):
    # В переменной track_list сохраняются те строки таблицы df, для которых 
    # значение в столбце 'day' равно параметру day и одновременно значение
    # в столбце 'city' равно параметру city (используйте последовательную фильтрацию
    # с помощью логической индексации).
    track_list = df[ (df.day == day) & (df.city == city) ]
    
    # В переменной track_list_count сохраняется число значений столбца 'user_id',
    # рассчитанное методом count() для таблицы track_list.
    # Функция возвращает число - значение track_list_count.
    track_list_count = track_list.user_id.count()
    
    return track_list_count

Вызовите `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [23]:
# количество прослушиваний в Москве по понедельникам
number_tracks( 'Monday', 'Moscow' )

NameError: name 'df' is not defined

In [24]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks( 'Monday', 'Saint-Petersburg' )

NameError: name 'df' is not defined

In [25]:
# количество прослушиваний в Москве по средам
number_tracks( 'Wednesday', 'Moscow' )

NameError: name 'df' is not defined

In [26]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks( 'Wednesday', 'Saint-Petersburg' )

NameError: name 'df' is not defined

In [27]:
# количество прослушиваний в Москве по пятницам
number_tracks( 'Friday', 'Moscow' )

NameError: name 'df' is not defined

In [28]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks( 'Friday', 'Saint-Petersburg' )

NameError: name 'df' is not defined

Создайте c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [29]:
# Таблица с результатами
output = pd.DataFrame( data={'city':['Moscow', 'Saint-Petersburg'],
                             'monday':[number_tracks( 'Monday', 'Moscow' ), number_tracks( 'Monday', 'Saint-Petersburg' )], 
                             'wednesday':[number_tracks( 'Wednesday', 'Moscow' ), number_tracks( 'Wednesday', 'Saint-Petersburg' )],
                             'friday':[number_tracks( 'Friday', 'Moscow' ), number_tracks( 'Friday', 'Saint-Petersburg' )]} )
display( output )

NameError: name 'df' is not defined

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраните таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [30]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[ df.city == 'Moscow' ]

NameError: name 'df' is not defined

In [31]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[ df.city == 'Saint-Petersburg' ]

NameError: name 'df' is not defined

Создайте функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [32]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в
# заданное время:
def genre_weekday( table, day, time1, time2 ):
    # 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
    #    которых одновременно:
    #    - значение в столбце day равно значению аргумента day
    #    - значение в столбце time больше значения аргумента time1
    #    - значение в столбце time меньше значения аргумента time2
    #    Используйте последовательную фильтрацию с помощью логической индексации.
    genre_df = table[ (table.day == day) & (table.time >= time1) & (table.time <= time2) ]

    # 2) сгруппировать датафрейм genre_df по столбцу genre, взять один из его
    #    столбцов и посчитать методом count() количество записей для каждого из
    #    присутствующих жанров, получившийся Series записать в переменную
    #    genre_df_count
    genre_df_count = genre_df.groupby('genre').user_id.count()
    
    # 3) отсортировать genre_df_count по убыванию встречаемости и сохранить
    #    в переменную genre_df_sorted
    genre_df_sorted = genre_df_count.sort_values( ascending=False )
    
    # 4) вернуть Series из 10 первых значений genre_df_sorted, это будут топ-10
    #    популярных жанров (в указанный день, в заданное время)
    return genre_df_sorted[:10]

Cравните результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [33]:
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
# объекты, хранящие время, являются строками и сравниваются как строки
# пример вызова: genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

NameError: name 'moscow_general' is not defined

In [34]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

NameError: name 'spb_general' is not defined

In [35]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

NameError: name 'moscow_general' is not defined

In [36]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

NameError: name 'spb_general' is not defined

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируйте таблицу `moscow_general` по жанру и посчитайте прослушивания треков каждого жанра методом `count()`. Затем отсортируйте результат в порядке убывания и сохраните его в таблице `moscow_genres`.

In [37]:
# одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres
moscow_genres = moscow_general.groupby('genre').genre.count().sort_values( ascending=False )

NameError: name 'moscow_general' is not defined

Выведите на экран первые десять строк `moscow_genres`:

In [38]:
# просмотр первых 10 строк moscow_genres
moscow_genres.head( 10 )

NameError: name 'moscow_genres' is not defined

Теперь повторите то же и для Петербурга.

Сгруппируйте таблицу `spb_general` по жанру. Посчитайте прослушивания треков каждого жанра. Результат отсортируйте в порядке убывания и сохраните в таблице `spb_genres`:


In [39]:
# одной строкой: группировка таблицы spb_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres
spb_genres = spb_general.groupby('genre').genre.count().sort_values( ascending=False )

NameError: name 'spb_general' is not defined

Выведите на экран первые десять строк `spb_genres`:

In [40]:
# просмотр первых 10 строк spb_genres
spb_genres.head( 10 )

NameError: name 'spb_genres' is not defined

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Вы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 
С методами проверок гипотез вы ещё познакомитесь в следующих темах.